In [ ]:
import os
import pandas as pd
import numpy as np
import mne
from mne.preprocessing import ICA, create_ecg_epochs, create_eog_epochs
from mne.filter import filter_data, notch_filter
import pickle

# 폴더 경로 설정
eeg_data_path = r"C:\Users\User\Documents\Lie detect data\모의범죄 실험 데이터\EEG 데이터3"
code_data_path = r'C:\Users\User\Documents\Lie detect data\모의범죄 실험 데이터\행동 데이터2\data'
truth_file_path = r'C:\Users\User\Documents\Lie detect data\모의범죄 실험 데이터\행동 데이터2\choice&response.xlsx'
output_data_path = r'C:\Users\User\Documents\Lie detect data\npy 데이터3'

# 폴더와 파일 존재 확인
if not os.path.exists(eeg_data_path):
    raise FileNotFoundError(f"EEG 데이터 폴더가 존재하지 않습니다: {eeg_data_path}")
if not os.path.exists(code_data_path):
    raise FileNotFoundError(f"행동 데이터 폴더가 존재하지 않습니다: {code_data_path}")
if not os.path.exists(truth_file_path):
    raise FileNotFoundError(f"진실/거짓 정보 파일이 존재하지 않습니다: {truth_file_path}")
if not os.path.exists(output_data_path):
    os.makedirs(output_data_path)

# 진실/거짓 정보 파일 읽기
truth_df = pd.read_excel(truth_file_path, header=None, index_col=None)
truth_df.columns = ['번호', '항목', '진실/거짓']
truth_info = {str(int(row['번호'])): row['진실/거짓'] == '진실' for idx, row in truth_df.iterrows()}

print("진실/거짓 정보 파일 로드 완료. 첫 5개 항목:")
print(truth_df.head())

# 피험자 파일 리스트
eeg_files = [f for f in os.listdir(eeg_data_path) if f.endswith('.mff')]
print(f"총 {len(eeg_files)}개의 EEG 파일을 발견했습니다.")

# 통합 데이터를 저장할 리스트 초기화
combined_eeg_data = {}
combined_poly_data = {}

def save_epochs(epochs_data, epoch_labels, data_type, participant_number, combined_data):
    labels = set(epoch_labels)
    for label in labels:
        label_indices = [i for i, l in enumerate(epoch_labels) if l == label]
        label_data = [epochs_data[i] for i in label_indices]
        
        # 개별 피험자 데이터 저장
        with open(os.path.join(output_data_path, f"{data_type}_{label}_{participant_number}.pkl"), 'wb') as f:
            pickle.dump(label_data, f)
        
        # 통합 데이터 업데이트
        if label not in combined_data:
            combined_data[label] = label_data
        else:
            combined_data[label].extend(label_data)
    
    print(f"{participant_number}의 {data_type} 데이터가 저장되었습니다.")

def preprocess_and_save_poly(raw_poly, events, event_labels, participant_number):
    # EDA 채널 전처리 (0.5~15 Hz, 60 Hz 노치 필터)
    eda_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['EDA'])
    raw_poly._data[eda_channel, :] = filter_data(raw_poly._data[eda_channel, :], raw_poly.info['sfreq'], l_freq=0.5, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[eda_channel, :] = notch_filter(raw_poly._data[eda_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # PPG 채널 전처리 (0.01~15 Hz, 60 Hz 노치 필터)
    ppg_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['PPG'])
    raw_poly._data[ppg_channel, :] = filter_data(raw_poly._data[ppg_channel, :], raw_poly.info['sfreq'], l_freq=0.01, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[ppg_channel, :] = notch_filter(raw_poly._data[ppg_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # Resp. Effort 채널 전처리 (1~15 Hz, 60 Hz 노치 필터)
    resp_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['Resp. Effort'])
    raw_poly._data[resp_channel, :] = filter_data(raw_poly._data[resp_channel, :], raw_poly.info['sfreq'], l_freq=1, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[resp_channel, :] = notch_filter(raw_poly._data[resp_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # ECG 채널 전처리 (1~70 Hz, 60 Hz 노치 필터)
    ecg_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['ECG'])
    raw_poly._data[ecg_channel, :] = filter_data(raw_poly._data[ecg_channel, :], raw_poly.info['sfreq'], l_freq=1, h_freq=70, method='fir', fir_design='firwin')
    raw_poly._data[ecg_channel, :] = notch_filter(raw_poly._data[ecg_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # 폴리그래프 데이터 에포킹
    epochs_list = []
    epoch_labels = []
    for i in range(len(events)):
        event_id = events[i][2]
        onset = events[i][0]

        if event_id <= 6:
            if 'que' in event_labels[i]:
                try:
                    ans_event_index = next(j for j in range(i + 1, len(events)) if 'ans' in event_labels[j])
                    end_onset = events[ans_event_index][0]
                    epochs_list.append([onset, end_onset])
                    epoch_labels.append(event_labels[i])
                except StopIteration:
                    pass
            elif 'ans' in event_labels[i]:
                end_onset = onset + int(10 * raw_poly.info['sfreq'])
                epochs_list.append([onset, end_onset])
                epoch_labels.append(event_labels[i])

    epochs_data = []
    for start, end in epochs_list:
        if end <= len(raw_poly.times):
            epoch_data = raw_poly.get_data(start=start, stop=end)
            epochs_data.append(epoch_data)

    print(f"{participant_number}의 폴리그래프 데이터 에포킹 완료. 총 {len(epochs_data)}개의 에포크")
    save_epochs(epochs_data, epoch_labels, 'poly', participant_number, combined_poly_data)

def process_participant_data(participant_number):    
    eeg_file = next(f for f in eeg_files if participant_number in f)
    file_path = os.path.join(eeg_data_path, eeg_file)
    code_file = next(f for f in os.listdir(code_data_path) if participant_number in f)

    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        print(f"EEG 파일이 존재하지 않습니다: {file_path}")
        return
    if not os.path.exists(os.path.join(code_data_path, code_file)):
        print(f"행동 데이터 파일이 존재하지 않습니다: {code_file}")
        return

    # 데이터 로드
    raw = mne.io.read_raw_egi(file_path, preload=True)
    raw.resample(250)
    print(f"{participant_number}의 EEG 데이터 로드 및 리샘플링 완료")

    # EEG 채널과 폴리그래프 채널 분리
    eeg_channels = mne.pick_types(raw.info, eeg=True, stim=False, bio=False, ecg=False, eog=False)
    bio_channels = mne.pick_types(raw.info, eeg=False, stim=False, bio=True, ecg=True, eog=False)

    print(f"선택된 EEG 채널: {len(eeg_channels)}개, 선택된 Bio 채널: {len(bio_channels)}개")

    # EEG 데이터 전처리
    raw_eeg = raw.copy().pick(eeg_channels)
    raw_eeg.filter(0.1, 50, fir_design='firwin')
    raw_eeg.notch_filter(freqs=[60, 120], fir_design='firwin')
    raw_eeg.set_eeg_reference('average', projection=True)
    raw_eeg.apply_proj()
    print(f"{participant_number}의 EEG 데이터 전처리 완료")

    # ICA to remove ECG and EOG artifacts
    ica = ICA(n_components=15, random_state=97, max_iter=800)
    ica.fit(raw_eeg)

    # ECG 아티팩트를 찾고 제거
    ecg_epochs = create_ecg_epochs(raw, ch_name='ECG')
    ecg_indices, ecg_scores = ica.find_bads_ecg(ecg_epochs, method='correlation', threshold='auto')
    ica.exclude.extend(ecg_indices)

    # EOG 아티팩트를 수동으로 찾고 제거 (E1 채널 사용)
    eog_indices, eog_scores = ica.find_bads_eog(raw_eeg, ch_name='E1')
    ica.exclude.extend(eog_indices)

    # ICA 적용
    ica.apply(raw_eeg)
    print(f"{participant_number}의 ICA 적용 완료")

    # 폴리그래프 데이터 전처리
    raw_poly = raw.copy().pick(bio_channels)

    # EDA 채널 전처리 (0.5~15 Hz, 60 Hz 노치 필터)
    eda_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['EDA'])
    raw_poly._data[eda_channel, :] = filter_data(raw_poly._data[eda_channel, :], raw_poly.info['sfreq'], l_freq=0.5, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[eda_channel, :] = notch_filter(raw_poly._data[eda_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # PPG 채널 전처리 (0.01~15 Hz, 60 Hz 노치 필터)
    ppg_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['PPG'])
    raw_poly._data[ppg_channel, :] = filter_data(raw_poly._data[ppg_channel, :], raw_poly.info['sfreq'], l_freq=0.01, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[ppg_channel, :] = notch_filter(raw_poly._data[ppg_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # Resp. Effort 채널 전처리 (1~15 Hz, 60 Hz 노치 필터)
    resp_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['Resp. Effort'])
    raw_poly._data[resp_channel, :] = filter_data(raw_poly._data[resp_channel, :], raw_poly.info['sfreq'], l_freq=1, h_freq=15, method='fir', fir_design='firwin')
    raw_poly._data[resp_channel, :] = notch_filter(raw_poly._data[resp_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # ECG 채널 전처리 (1~70 Hz, 60 Hz 노치 필터)
    ecg_channel = mne.pick_channels(raw_poly.info['ch_names'], include=['ECG'])
    raw_poly._data[ecg_channel, :] = filter_data(raw_poly._data[ecg_channel, :], raw_poly.info['sfreq'], l_freq=1, h_freq=70, method='fir', fir_design='firwin')
    raw_poly._data[ecg_channel, :] = notch_filter(raw_poly._data[ecg_channel, :], raw_poly.info['sfreq'], freqs=60, method='fir', fir_design='firwin')

    # 이벤트 마커 추출
    events = mne.find_events(raw)

    # 이벤트 ID 매핑 생성
    event_mapping = {}
    event_counter = 1

    # 코드와 코드2 열의 값을 이벤트 ID와 매핑
    code_df = pd.read_csv(os.path.join(code_data_path, code_file))
    print(f"{participant_number}의 행동 데이터 로드 완료. 첫 5개 행:")
    print(code_df.head())

    for _, row in code_df.iterrows():
        code_value = row['code']
        code2_value = row['code2']

        if pd.notna(code_value) and code_value not in event_mapping:
            event_mapping[code_value] = event_counter
            event_counter += 1

        if pd.notna(code2_value) and code2_value not in event_mapping:
            event_mapping[code2_value] = event_counter
            event_counter += 1

    # quer와 ansr 이벤트의 인덱스를 추적
    quer_count = 0
    ansr_count = 0

    # 이벤트 라벨링
    event_labels = []
    row_index = 0
    is_code = True  # True면 code 열을, False면 code2 열을 참조

    for event in events:
        event_id = event[2]
        label = None

        if row_index < len(code_df):
            row = code_df.iloc[row_index]
            participant_number_split = str(int(participant_number.split('_')[1]))  # 피험자 번호 추출 및 앞의 0 제거
            is_truth = truth_info.get(participant_number_split, False)

            # code와 code2를 순차적으로 참조
            current_code = row['code'] if is_code else row['code2']

            if is_truth:
                if current_code == 'quer':
                    label = 'quer_truth'
                elif current_code == 'ansr':
                    label = 'ansr_truth'
                else:
                    label = current_code
            else:
                if current_code == 'quer':
                    quer_count += 1
                    if quer_count in [3, 4]:
                        quer_keys = row['key_resp_8.keys'].split(',')
                        if 'j' in quer_keys:
                            label = 'quer_truth'
                        else:
                            label = 'quer_lie'
                    else:
                        label = 'quer_lie'
                elif current_code == 'ansr':
                    ansr_count += 1
                    if ansr_count in [3, 4]:
                        ansr_keys = row['key_resp_8.keys'].split(',')
                        if 'j' in ansr_keys:
                            label = 'ansr_truth'
                        else:
                            label = 'ansr_lie'
                    else:
                        label = 'ansr_lie'
                else:
                    label = current_code

            # 다음 이벤트에서는 code2 열을 참조
            if is_code:
                is_code = False
            else:
                row_index += 1
                is_code = True

        event_labels.append(label)

    print(f"{participant_number}의 이벤트 라벨링 완료. 첫 10개 이벤트:")
    print(event_labels[:10])

    # 에포크 생성
    epochs_list = []
    epoch_labels = []
    for i in range(len(events)):
        event_id = events[i][2]
        onset = events[i][0]

        # 1, 2세트 이벤트 (1~6번 이벤트)만 사용
        if event_id <= 6:
            # que 이벤트인 경우
            if 'que' in event_labels[i]:
                try:
                    ans_event_index = next(j for j in range(i + 1, len(events)) if 'ans' in event_labels[j])
                    end_onset = events[ans_event_index][0]
                    epochs_list.append([onset, end_onset])
                    epoch_labels.append(event_labels[i])
                except StopIteration:
                    pass

            # ans 이벤트인 경우
            elif 'ans' in event_labels[i]:
                end_onset = onset + int(10 * raw.info['sfreq'])
                epochs_list.append([onset, end_onset])
                epoch_labels.append(event_labels[i])

    # 에포크 생성
    epochs_data = []
    for start, end in epochs_list:
        if end <= len(raw_eeg.times):
            epoch_data = raw_eeg.get_data(start=start, stop=end)
            epochs_data.append(epoch_data)
            print(f"Epoch from {start} to {end} with shape: {epoch_data.shape}")

    # EEG 데이터 저장
    save_epochs(epochs_data, epoch_labels, 'eeg', participant_number, combined_eeg_data)

    # 폴리그래프 데이터 저장
    preprocess_and_save_poly(raw_poly, events, event_labels, participant_number)

    print(f"Participant {participant_number} data processing complete.")

# 모든 피험자에 대해 반복 처리
for eeg_file in eeg_files:
    # 피험자 번호 추출
    participant_number = "_".join(eeg_file.split('_')[1:3])
    
    # 데이터 처리 함수 호출
    process_participant_data(participant_number)

# 통합 데이터 저장
for label, data in combined_eeg_data.items():
    print(f"Combining EEG data for label {label} with {len(data)} epochs")
    np.save(os.path.join(output_data_path, f"combined_eeg_{label}.npy"), np.array(data, dtype=object))

for label, data in combined_poly_data.items():
    print(f"Combining Poly data for label {label} with {len(data)} epochs")
    np.save(os.path.join(output_data_path, f"combined_poly_{label}.npy"), np.array(data, dtype=object))

print("모든 피험자 데이터 처리가 완료되었습니다.")